# Dependencies

In [3]:
!pip install monthdelta

In [22]:
# Import Package
import bs4 as bs
import urllib.request
import pandas as pd
import time
import datetime
import monthdelta
from config import sql_Password
from sqlalchemy import create_engine, Column, Integer, String, Float, Date

In [5]:
# Chrome Driver
# executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
# browser = Browser('chrome',**executable_path,headless=False)

# Web Scraping Yahoo

## Web Scraping Function
* Input: 
    * a) Ticker Name, 
    * b) Start date, in datetime format; 
    * c) End date, in datetime format; 
    * d) Optional, data frequency:
        * d.1) 1d (every business day); 
        * d.2) 1wk (every week); 
        * d.3) 1mo (every month)
* Output: Dataframe with the following column names:
    * 1) Open Price
    * 2) High Price
    * 3) Low Price
    * 4) Close Price
    * 5) Adj Close Price
    * 6) Volume

In [6]:
def get_historical_price(ticker, date1, date2, frequency='1d',display=True):
 
    format_string='%Y-%m-%d %H:%M:%S'
 
    # One day (86400 second) adjustment required to get dates printed to match web site manual output
    _date1 = date1.strftime("%Y-%m-%d 00:00:00")
    date1_epoch = str(int(time.mktime(time.strptime(_date1, format_string)))- 86400)
    
    if display == True: 
        print("")
        print(date1, date1_epoch, " + 86,400 = ", str(int(date1_epoch) + 86400))
 
    _date2 = date2.strftime("%Y-%m-%d 00:00:00")
    date2_epoch = str(int(time.mktime(time.strptime(_date2, format_string))))
    
    if display == True:
        print(date2, date2_epoch)
 
    url = 'https://finance.yahoo.com/quote/' + ticker + '/history?period1=' + date1_epoch + '&period2=' + date2_epoch + '&interval='+frequency+'&filter=history&frequency='+frequency
    source = urllib.request.urlopen(url).read()      
    soup = bs.BeautifulSoup(source,'lxml')
    table_rows = soup.find_all('tr')
      
    extract_table = []
      
    for table_row in table_rows:
        table_row_values = table_row.find_all('td')
        extract_row = [i.text for i in table_row_values]
        extract_table.append(extract_row)        
      
    column_names = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
  
    #extract_table = extract_table[1:-2]
    extract_table_df = pd.DataFrame(extract_table)
    extract_table_df.columns = column_names
    extract_table_df.set_index(column_names[0], inplace=True)
    extract_table_df = extract_table_df.convert_objects(convert_numeric=True)
    extract_table_df = extract_table_df.iloc[::-1]
    extract_table_df.dropna(inplace=True)
      
    return extract_table_df

## Web Scraping Function Trial with Apple Stock

In [7]:
## Try Scraping with APPL. 

# Initialize the end date to be today and start date is one year before. 
print("")
print("")
start_date = datetime.date(2018, 6, 2)
end_date = datetime.date(2019, 2, 19)
today = datetime.date.today()
 
# The statements in this group are for debugging purposes only
format_string='%Y-%m-%d %H:%M:%S'
t1 = start_date.strftime("%Y-%m-%d 00:00:00")
t2 = end_date.strftime("%Y-%m-%d 00:00:00")
start_date_epoch = str(int(time.mktime(time.strptime(t1, format_string))))
end_date_epoch = str(int(time.mktime(time.strptime(t2,format_string))))
 
# Output all 'original' dates
print('Today     :', today)
print('Start Date:', start_date, 'Start Date Epoch:', start_date_epoch)
print('End   Date:', end_date,   'End   Date Epoch:', end_date_epoch)
 
# Initialize 'date1'
date1 = start_date
 
# Initialize 'date1'
date1 = start_date
 
# Do not allow the 'End Date' to be AFTER today
if today < end_date:
    end_date = today

iteration_number = 0
while date1 <= end_date:
    iteration_number += 1
 
    # Create 'date2' in a 60 day Window or less
    date2 = date1 + monthdelta.monthdelta(2)
    date2 = datetime.date(date2.year, date2.month, 1)
    date2 = date2 - datetime.timedelta(days=1)
         
    # Do not allow 'date2' to go beyond the 'End Date'
    if date2 > end_date:
        date2 = end_date
         
    print(f"Processing {date1} thru {date2}.")
    stock_symbol = 'AAPL'
    df = get_historical_price(stock_symbol, date1, date2)
     
    if iteration_number == 1:
        dfall = df.copy()
    else:
        frames = [dfall, df]
        dfall = pd.concat(frames)
 
    # # # print(dfall)
    # # # print("len of dfall = {}".format(len(dfall)))
 
    # Increment the first date for the next pass
    date1 = date1   + monthdelta.monthdelta(2)
    date1 = datetime.date(date1.year, date1.month, 1)

# Output concatenated data set
print(dfall)
print(f"len of whole extracted data set = {len(dfall)}")



Today     : 2019-02-22
Start Date: 2018-06-02 Start Date Epoch: 1527915600
End   Date: 2019-02-19 End   Date Epoch: 1550556000
Processing 2018-06-02 thru 2018-07-31.

2018-06-02 1527829200  + 86,400 =  1527915600
2018-07-31 1533013200


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Processing 2018-08-01 thru 2018-09-30.

2018-08-01 1533013200  + 86,400 =  1533099600
2018-09-30 1538283600
Processing 2018-10-01 thru 2018-11-30.

2018-10-01 1538283600  + 86,400 =  1538370000
2018-11-30 1543557600
Processing 2018-12-01 thru 2019-01-31.

2018-12-01 1543557600  + 86,400 =  1543644000
2019-01-31 1548914400
Processing 2019-02-01 thru 2019-02-19.

2019-02-01 1548914400  + 86,400 =  1549000800
2019-02-19 1550556000
                Open    High     Low   Close  Adj Close      Volume
Date                                                               
Jun 01, 2018  187.99  190.26  187.75  190.24     188.11  23,442,500
Jun 04, 2018  191.64  193.42  191.35  191.83     189.68  26,266,200
Jun 05, 2018  193.07  193.94  192.36  193.31     191.14  21,566,000
Jun 06, 2018  193.63  194.08  191.92  193.98     191.81  20,933,600
Jun 07, 2018  194.14  194.20  192.34  193.46     191.29  21,347,200
Jun 08, 2018  191.17  192.00  189.77  191.70     189.55  26,656,800
Jun 11, 2018  191.35  19

## Web Scraping with Short and Long Term Bond ETFs

### Function to scrape long term data

In [8]:
def get_long_historical_price(ticker, start_date, end_date, frequency='1d',display=True):

    date1 = start_date
    iteration_number = 1

    dfall = {}
    
    while date1 <= end_date:

        if frequency == '1d':      
            month_delta = 3         
        else:          
            if frequency == '1wk':            
                month_delta = 12            
            else:            
                month_delta = 48
                                
        # Create 'date2' in a 60 day Window or less
        date2 = date1 + monthdelta.monthdelta(month_delta)
        date2 = datetime.date(date2.year, date2.month, 1)
        date2 = date2 - datetime.timedelta(days = 1)

        # Do not allow 'date2' to go beyond the 'End Date'
        if date2 > end_date:
            date2 = end_date
            
        # print(f"Processing {date1} thru {date2}.")

        try: 
        
            df = get_historical_price(ticker, date1, date2, frequency, display)  

            if len(df) > 0: 
                
                if iteration_number == 1:
                    dfall = df.copy()
                else:
                    frames = [dfall, df]
                    dfall = pd.concat(frames) 

                iteration_number += 1 
                
            date1 = date1 + monthdelta.monthdelta(month_delta)
            date1 = datetime.date(date1.year, date1.month, 1)
            
        except:
            
            date1 = date1 + monthdelta.monthdelta(month_delta)
            date1 = datetime.date(date1.year, date1.month, 1)
            
    return dfall

### ETF Tickers
* Long Term Bond ETFs:
    * IEF --> iShares Barclays 7-10 Year Trasry Bnd Fd
    * DTYL --> BARCLAY BK IPAT US TR 10 YR BULL ETN
    * EDV --> VANGUARD WORLD/EXTD DURATION TREAS
    * TLH --> iShares 10-20 Year Treasury Bond ETF
* Long Term Bond ETFs:
    * SHV --> iShares Short Treasury Bond ETF 
    * VGSH --> Vanguard Short-Term Treasury ETF
    * SCHR --> Schwab Intermediate-Term US Trs ETF

In [9]:
Short_TR_Tickers = ['IEF','DTYL','EDV','TLH']
Long_TR_Tickers = ['SHV','VGSH','SCHR']

### Loop through each ticker to scrape data, Scrape from 2000 to today.  

In [10]:
# Scrape Short Term Bonds
Short_TR_Data_Dict = {}

for ticker in Short_TR_Tickers:
    
    # Set up start and end date
    start_date = datetime.date(2000, 1, 2)
    end_date = datetime.date.today()
    
    print("")
    print("")
    print (f'Scraping ticker is {ticker}')
   
    ticker_df = get_long_historical_price(ticker, start_date, end_date, '1d', False)
    
    print ('Successfully Scraping historical data from ',ticker_df.index[0],' to ',ticker_df.index[-1])
    print (f'Number of data extracted = {len(ticker_df)}')
    
    Short_TR_Data_Dict = {**Short_TR_Data_Dict,**{ticker:ticker_df}}



Scraping ticker is IEF


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Successfully Scraping historical data from  Jul 30, 2002  to  Feb 22, 2019
Number of data extracted = 4219


Scraping ticker is DTYL
Successfully Scraping historical data from  Aug 11, 2010  to  Jan 30, 2019
Number of data extracted = 430


Scraping ticker is EDV
Successfully Scraping historical data from  Jan 31, 2008  to  Feb 22, 2019
Number of data extracted = 2633


Scraping ticker is TLH
Successfully Scraping historical data from  Jan 12, 2007  to  Feb 22, 2019
Number of data extracted = 2854


In [11]:
# Scrape Short Term Bonds
Long_TR_Data_Dict = {}

for ticker in Long_TR_Tickers:
    
    # Set up start and end date
    start_date = datetime.date(2000, 1, 2)
    end_date = datetime.date.today()
    
    print("")
    print("")
    print (f'Scraping ticker is {ticker}')
   
    ticker_df = get_long_historical_price(ticker, start_date, end_date, '1d', False)
    
    print ('Successfully Scraping historical data from ',ticker_df.index[0],' to ',ticker_df.index[-1])
    print (f'Number of data extracted = {len(ticker_df)}')
    
    Long_TR_Data_Dict = {**Long_TR_Data_Dict,**{ticker:ticker_df}}



Scraping ticker is SHV


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Successfully Scraping historical data from  Jan 11, 2007  to  Feb 22, 2019
Number of data extracted = 3030


Scraping ticker is VGSH
Successfully Scraping historical data from  Dec 31, 2009  to  Feb 22, 2019
Number of data extracted = 2201


Scraping ticker is SCHR
Successfully Scraping historical data from  Aug 05, 2010  to  Feb 22, 2019
Number of data extracted = 2174


## AGGREGATE DATA

### Aggregate and Clean Data

In [12]:
total_Data_Dict = {**Short_TR_Data_Dict,**Long_TR_Data_Dict}

In [13]:
total_Data_Dict['SHV'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
"Jan 11, 2007",108.70,108.70,108.70,108.70,98.78,900
"Jan 16, 2007",108.76,108.76,108.74,108.74,98.81,500
"Apr 02, 2007",108.99,109.03,108.91,109.00,99.83,"23,000"
"Apr 03, 2007",109.05,109.07,108.98,109.02,99.85,"25,000"
"Apr 04, 2007",109.07,109.11,109.06,109.08,99.90,"20,800"


In [14]:
'''
Go Through each table and do the followings:
1) Adjust column names to replace space with _
2) Change all the data from string to float
3) Add extra column with the ticker name
4) Change the date string to datetime object
'''
for key,table in total_Data_Dict.items():
    table.rename(columns={'Adj Close':'Adj_Close'},inplace=True)
    table_name = table.columns
    for name in table_name:
        table[name] = table[name].apply(lambda x: float(x.replace(',','').replace('-','0')) if isinstance(x,str) else x)
    table = table.reset_index()
    table['Ticker'] = key
    table['Date'] = table['Date'].apply(lambda x: datetime.datetime.strptime(x,'%b %d, %Y').date())
    total_Data_Dict[key] = table

In [15]:
# Display the cleaned up date
total_Data_Dict['SHV'].head()

,Date,Open,High,Low,Close,Adj_Close,Volume,Ticker
0,2007-01-11,108.70,108.70,108.70,108.70,98.78,900.0,SHV
1,2007-01-16,108.76,108.76,108.74,108.74,98.81,500.0,SHV
2,2007-04-02,108.99,109.03,108.91,109.00,99.83,23000.0,SHV
3,2007-04-03,109.05,109.07,108.98,109.02,99.85,25000.0,SHV
4,2007-04-04,109.07,109.11,109.06,109.08,99.90,20800.0,SHV


In [29]:
# Aggregate all the tables
iteration = 1
for key,table in total_Data_Dict.items():
    if iteration == 1:
        agg_table = table.copy()
    else:
        agg_table = pd.concat([agg_table,table],ignore_index=True)
    iteration += 1
    
agg_table.index.name = 'id'
agg_table.index += 1
    
print(f'The aggregated table has data = {len(agg_table)}')

The aggregated table has data = 17541


### Save Data to SQL

In [17]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [20]:
connection_string = "root:"+sql_Password+"@localhost/ETL_db"
engine = create_engine(f'mysql://{connection_string}')

In [23]:
# Define Yield Curve Class
class ETF_Data(Base):
    __tablename__ = 'ETF_Data'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    Ticker = Column(String(5))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)  
    Close = Column(Float) 
    Adj_Close = Column(Float)
    Volume = Column(Float)

In [24]:
Base.metadata.create_all(engine)

In [30]:
agg_table.to_sql(name='ETF_Data', con=engine, if_exists='append', index=True)